In [ ]:
#| default_exp hotspots

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export

from pathlib import Path
import multiprocessing
from collections import defaultdict

import numpy as np
from hdbscan import HDBSCAN

from clip_plot.images import ImageFactory
from clip_plot.utils import write_json, timestamp, get_json_path
from clip_plot.configuration import ClusterSpec

In [ ]:
#| export

def get_cluster_model(min_cluster_size: int = 15):
    """Return model with .fit() method that can be used to cluster input vectors
    """
    return HDBSCAN(
        core_dist_n_jobs=multiprocessing.cpu_count(),
        min_cluster_size=min_cluster_size,
        cluster_selection_epsilon=0.01,
        min_samples=1,
        approx_min_span_tree=False,
    )

In [ ]:
#| export

def get_hotspots(imageEngine: ImageFactory, vecs: np.ndarray,
                 data_dir: Path, plot_id: str,
                 cluster_spec: ClusterSpec,
                ):
    """Return the stable clusters from the condensed tree of connected components from the density graph

    Args:
        layouts (Optional[dict] = {}) 
        use_high_dimensional_vectors (Optional[bool] = True) 
        n_preproc_dims
        vecs
        umap = Used if use_high_dimensional_vectors is False
        max_clusters

    """
    print(timestamp(), "Clustering data with HDBSCAN")
    model = get_cluster_model(cluster_spec.min_cluster_size)
    z = model.fit(vecs)

    # create a map from cluster label to image indices in cluster
    d = defaultdict(lambda: defaultdict(list))
    for idx, i in enumerate(z.labels_):
        if i != -1:
            d[i]["images"].append(idx)
            d[i]["img"] = imageEngine[idx].filename
            d[i]["layout"] = "inception_vectors"

    # remove massive clusters
    deletable = []
    for i in d:
        # find percent of images in cluster
        image_percent = len(d[i]["images"]) / len(vecs)
        # determine if image or area percent is too large
        if image_percent > 0.5:
            deletable.append(i)
    for i in deletable:
        del d[i]

    # sort the clusers by size and then label the clusters
    clusters = d.values()
    clusters = sorted(clusters, key=lambda i: len(i["images"]), reverse=True)
    for idx, i in enumerate(clusters):
        i["label"] = f"Cluster {idx + 1}"

    # slice off the first `max_clusters`
    clusters = clusters[: cluster_spec.max_clusters]

    # save the hotspots to disk and return the path to the saved json
    print(timestamp(), "Found", len(clusters), "hotspots")
    return write_json(get_json_path(data_dir, "hotspots", "hotspot", plot_id),
                      clusters)

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()